# Importing libraries and reading data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
# import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('max_columns',200)

In [2]:
df = pd.read_csv('/kaggle/input/kagglex-bipoc-2022-2023-ml-foundation/Train.csv')

In [3]:
df.AAGE.describe()

count    157509.000000
mean         39.816112
std          19.404058
min           0.000000
25%          26.000000
50%          38.000000
75%          52.000000
max          90.000000
Name: AAGE, dtype: float64

In [23]:
df.age.unique()

array([22, 37, 83, 46, 35, 21, 67, 24, 42, 52, 26, 31, 30, 27, 18, 34, 45,
       36, 43, 38, 16, 25, 39, 29, 72, 32, 17, 57, 56, 15,  8, 20, 48, 64,
        1, 53, 33, 40,  7, 63, 44, 12, 74,  9, 62, 49, 50, 19, 58, 10,  4,
       60, 28, 86,  6, 76, 68, 41,  3, 23, 14, 65, 47, 51, 77, 66, 90, 73,
       54,  2, 89, 70,  0, 69, 84, 79, 61, 59, 11,  5, 71, 55, 81, 82, 13,
       75, 78, 87, 88, 80, 85])

In [4]:
df.AMJOCC.unique()

array([' Not in universe', ' Precision production craft & repair',
       ' Adm support including clerical', ' Sales',
       ' Machine operators assmblrs & inspctrs',
       ' Handlers equip cleaners etc ', ' Professional specialty',
       ' Private household services',
       ' Transportation and material moving',
       ' Executive admin and managerial',
       ' Technicians and related support', ' Other service',
       ' Farming forestry and fishing', ' Protective services',
       ' Armed Forces'], dtype=object)

In [5]:
df.query('AAGE <= 18')

,ID,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,ARACE,AREORGN,ASEX,AUNMEM,AUNTYPE,AWKSTAT,CAPGAIN,CAPLOSS,DIVVAL,FILESTAT,GRINREG,GRINST,HHDFMX,HHDREL,MIGMTR1,MIGMTR3,MIGMTR4,MIGSAME,MIGSUN,NOEMP,PARENT,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,YEAR,TARGET
15,90quolmt8w3e,18,Private,33,41,11th grade,0,High school,Never married,Retail trade,Handlers equip cleaners etc,White,NA,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Single,Midwest,Ohio,Child 18+ never marr Not in a subfamily,Child 18 or older,MSA to MSA,Same county,Same county,No,No,4,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,48,1994,0
24,ju9k9hnzqpa3,16,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,1,Mother only present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,12,1994,0
32,dk3zj1bfjeuj,17,Private,2,44,10th grade,0,High school,Never married,Agriculture,Farming forestry and fishing,Black,All other,Male,Not in universe,Not in universe,Full-time schedules,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,NaN,NaN,NaN,Not in universe under 1 year old,NaN,6,Mother only present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,5,1995,0
34,rm6f1h50n56r,16,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,1,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,7,1994,0
39,st0wocmgyvvy,15,Not in universe,0,0,10th grade,0,Not in universe,Never married,Not in universe or children,Not in universe,White,Puerto Rican,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Mother only present,Puerto-Rico,Puerto-Rico,United-States,Native- Born in the United States,0,Not in universe,2,0,1994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157480,ydz28su51sri,16,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,White,All other,Male,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Both parents present,United-States,Scotland,United-States,Native- Born in the United States,0,Not in universe,2,0,1995,0
157490,w2ten9motgni,17,Not in universe,0,0,5th or 6th grade,0,Not in universe,Never married,Not in universe or children,Not in universe,Black,Central or South American,Male,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Mother only present,Dominican-Republic,Dominican-Republic,Dominican-Republic,Foreign born- Not a citizen of U S,0,Not in universe,2,0,1995,0
157501,7q0v9obr5wbf,1,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,Asi

# Step 1: Data Understanding
- Dataframe `shape`
- `head` and `tail`
- `dtypes`
- `describe`

1. make the column names understandable by renaming them and updating the dataframe
2. learn which columns to keep and remove
3. convert object types to categorical or whatever dtype suitable for it

In [6]:
df.shape

(157509, 42)

In [7]:
df.head()

,ID,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,ARACE,AREORGN,ASEX,AUNMEM,AUNTYPE,AWKSTAT,CAPGAIN,CAPLOSS,DIVVAL,FILESTAT,GRINREG,GRINST,HHDFMX,HHDREL,MIGMTR1,MIGMTR3,MIGMTR4,MIGSAME,MIGSUN,NOEMP,PARENT,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,YEAR,TARGET
0,shydl6zxik4o,22,Not in universe,0,0,High school graduate,0,Not in universe,Divorced,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Single,Not in universe,Not in universe,Secondary individual,Nonrelative of householder,NaN,NaN,NaN,Not in universe under 1 year old,NaN,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,36,1995,0
1,2dhm421r62rr,37,Private,35,33,Some college but no degree,0,Not in universe,Divorced,Finance insurance and real estate,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Full-time schedules,0,0,0,Single,Not in universe,Not in universe,Nonfamily householder,Householder,NaN,NaN,NaN,Not in universe under 1 year old,NaN,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,47,1995,0
2,ys7nzza2hffj,83,Not in universe,0,0,1st 2nd 3rd or 4th grade,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,Puerto Rican,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Nonfamily householder,Householder,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Not in universe,Puerto-Rico,Puerto-Rico,Puerto-Rico,Native- Born in Puerto Rico or U S Outlying,0,Not in universe,2,0,1995,0
3,iqrpxh3sr7n1,46,Private,32,35,High school graduate,0,Not in universe,Married-civilian spouse present,Wholesale trade,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Full-time schedules,0,0,1800,Joint both under 65,Not in universe,Not in universe,Householder,Householder,NaN,NaN,NaN,Not in universe under 1 year old,NaN,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,1995,0
4,rr8vlzk9iwyn,35,Private,34,26,Associates degree-academic program,0,Not in universe,Married-civilian spouse present,Finance insurance and real estate,Adm support including clerical,White,All other,Female,Not in universe,Not in universe,Full-time schedules,0,0,0,Joint both under 65,Not in universe,Not in universe,Spouse of householder,Spouse of householder,NaN,NaN,NaN,Not in universe under 1 year old,NaN,6,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,45,1995,0


In [8]:
# df.tail()

In [9]:
df.dtypes

ID          object
AAGE         int64
ACLSWKR     object
ADTIND       int64
ADTOCC       int64
AHGA        object
AHRSPAY      int64
AHSCOL      object
AMARITL     object
AMJIND      object
AMJOCC      object
ARACE       object
AREORGN     object
ASEX        object
AUNMEM      object
AUNTYPE     object
AWKSTAT     object
CAPGAIN      int64
CAPLOSS      int64
DIVVAL       int64
FILESTAT    object
GRINREG     object
GRINST      object
HHDFMX      object
HHDREL      object
MIGMTR1     object
MIGMTR3     object
MIGMTR4     object
MIGSAME     object
MIGSUN      object
NOEMP        int64
PARENT      object
PEFNTVTY    object
PEMNTVTY    object
PENATVTY    object
PRCITSHP    object
SEOTR        int64
VETQVA      object
VETYN        int64
WKSWORK      int64
YEAR         int64
TARGET       int64
dtype: object

In [10]:
df.describe()

,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,NOEMP,SEOTR,VETYN,WKSWORK,YEAR,TARGET
count,157509.000000,157509.00000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000,157509.000000
mean,39.816112,20.30844,14.971830,72.889797,577.585992,49.300015,264.806386,2.580913,0.234825,1.834517,30.600182,1994.490893,0.082408
std,19.404058,18.18385,14.890825,311.546534,5403.024319,311.199032,2282.967898,2.400063,0.629650,0.538978,23.635018,0.499919,0.274986
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1994.000000,0.000000
25%,26.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1994.000000,0.000000
50%,38.000000,24.00000,12.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,48.000000,1994.000000,0.000000
75%,52.000000,37.00000,29.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,2.000000,52.000000,1995.000000,0.000000
max,90.000000,51.00000,46.000000,9916.000000,99999.000000,4608.000000,99999.000000,6.000000,2.000000,2.000000,52.000000,1995.000000,1.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157509 entries, 0 to 157508
Data columns (total 42 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        157509 non-null  object
 1   AAGE      157509 non-null  int64 
 2   ACLSWKR   157509 non-null  object
 3   ADTIND    157509 non-null  int64 
 4   ADTOCC    157509 non-null  int64 
 5   AHGA      157509 non-null  object
 6   AHRSPAY   157509 non-null  int64 
 7   AHSCOL    157509 non-null  object
 8   AMARITL   157509 non-null  object
 9   AMJIND    157509 non-null  object
 10  AMJOCC    157509 non-null  object
 11  ARACE     157509 non-null  object
 12  AREORGN   157509 non-null  object
 13  ASEX      157509 non-null  object
 14  AUNMEM    157509 non-null  object
 15  AUNTYPE   157509 non-null  object
 16  AWKSTAT   157509 non-null  object
 17  CAPGAIN   157509 non-null  int64 
 18  CAPLOSS   157509 non-null  int64 
 19  DIVVAL    157509 non-null  int64 
 20  FILESTAT  157509 non-null 

## Renaming Column names

In [12]:
df.columns

Index(['ID', 'AAGE', 'ACLSWKR', 'ADTIND', 'ADTOCC', 'AHGA', 'AHRSPAY',
       'AHSCOL', 'AMARITL', 'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX',
       'AUNMEM', 'AUNTYPE', 'AWKSTAT', 'CAPGAIN', 'CAPLOSS', 'DIVVAL',
       'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1',
       'MIGMTR3', 'MIGMTR4', 'MIGSAME', 'MIGSUN', 'NOEMP', 'PARENT',
       'PEFNTVTY', 'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'SEOTR', 'VETQVA',
       'VETYN', 'WKSWORK', 'YEAR', 'TARGET'],
      dtype='object')

In [13]:
df.rename(columns = {
#             'ID': , 
            'AAGE': 'age', 
            'ACLSWKR': 'class of worker', 
            'ADTIND': 'industry code', 
            'ADTOCC': 'occupation code', 
            'AHGA':'education', 
            'AHRSPAY':'wage per hour',
            'AHSCOL':'enrolled in edu inst last wk', 
            'AMARITL':'marital status', 
            'AMJIND':'major industry code', 
            'AMJOCC':'major occupation code', 
            'ARACE':'race', 
            'AREORGN':'hispanic origin', 
            'ASEX':'gender',
            'AUNMEM':'member of a labor union', 
            'AUNTYPE':'reason for unemployment', 
            'AWKSTAT':'full or part time employment stat', 
            'CAPGAIN':'capital gains', 
            'CAPLOSS':'capital losses', 
            'DIVVAL':'divdends from stocks',
            'FILESTAT': 'tax filer status', 
            'GRINREG':'region of previous residence', 
            'GRINST':'state of previous residence', 
            'HHDFMX':'detailed household and family stat', 
            'HHDREL':'detailed household summary in household', 
            'MIGMTR1':'migration code-change in msa',
            'MIGMTR3':'migration code-change in reg', 
            'MIGMTR4':'migration code-move within reg', 
            'MIGSAME':'live in this house 1 year ago', 
            'MIGSUN':'migration prev res in sunbelt', 
            'NOEMP':'num persons worked for employer', 
            'PARENT':'family members under 18',
            'PEFNTVTY':'country of birth father', 
            'PEMNTVTY':'country of birth mother', 
            'PENATVTY':'country of birth self', 
            'PRCITSHP':'citizenship', 
            'SEOTR':'own business or self employed', 
            'VETQVA':'fill inc questionnaire for veteran\'s admin',
            'VETYN':'veterans benefits', 
            'WKSWORK':'weeks worked in year', 
#             'YEAR', 
#             'TARGET'    
}, inplace=True)

In [14]:
df.columns

Index(['ID', 'age', 'class of worker', 'industry code', 'occupation code',
       'education', 'wage per hour', 'enrolled in edu inst last wk',
       'marital status', 'major industry code', 'major occupation code',
       'race', 'hispanic origin', 'gender', 'member of a labor union',
       'reason for unemployment', 'full or part time employment stat',
       'capital gains', 'capital losses', 'divdends from stocks',
       'tax filer status', 'region of previous residence',
       'state of previous residence', 'detailed household and family stat',
       'detailed household summary in household',
       'migration code-change in msa', 'migration code-change in reg',
       'migration code-move within reg', 'live in this house 1 year ago',
       'migration prev res in sunbelt', 'num persons worked for employer',
       'family members under 18', 'country of birth father',
       'country of birth mother', 'country of birth self', 'citizenship',
       'own business or self employ

# Step 2: Data Preperation
- Dropping irrelevant columns and rows
- Identifying duplicated columns
- Renaming Columns
- Feature Creation

## Subsetting colums to prepare for training

In [ ]:
m = ['ID', 'age', 'class of worker', 'industry code', 'occupation code',
       'education', 'wage per hour', 'enrolled in edu inst last wk',
       'marital status', 'major industry code', 'major occupation code',
       'race', 'hispanic origin', 'gender', 'member of a labor union',
       'reason for unemployment', 'full or part time employment stat',
       'capital gains', 'capital losses', 'divdends from stocks',
       'tax filer status', 'region of previous residence',
       'state of previous residence', 'detailed household and family stat',
       'detailed household summary in household',
       'migration code-change in msa', 'migration code-change in reg',
       'migration code-move within reg', 'live in this house 1 year ago',
       'migration prev res in sunbelt', 'num persons worked for employer',
       'family members under 18', 'country of birth father',
       'country of birth mother', 'country of birth self', 'citizenship',
       'own business or self employed',
       'fill inc questionnaire for veteran\'s admin', 'veterans benefits',
       'weeks worked in year', 'YEAR', 'TARGET']
for i in m:
    print(i+'\n')
#     print(df[i].unique())
#     print('\n'+'*'*20+'\n')
    print(df[i].value_counts())
    print('\n'+'*'*20+'\n')

## Figuring out which column to drop

In [16]:
# Example of dropping columns
# df.drop(['YEAR'], axis=1)

In [17]:
df.isna().sum()

ID                                                0
age                                               0
class of worker                                   0
industry code                                     0
occupation code                                   0
education                                         0
wage per hour                                     0
enrolled in edu inst last wk                      0
marital status                                    0
major industry code                               0
major occupation code                             0
race                                              0
hispanic origin                                   0
gender                                            0
member of a labor union                           0
reason for unemployment                           0
full or part time employment stat                 0
capital gains                                     0
capital losses                                    0
divdends fro

In [18]:
df = df[['ID', 'age', 
         #'class of worker', 'industry code', 'occupation code',
       'education', 
         #'wage per hour', 'enrolled in edu inst last wk',
       #'marital status', 'major industry code', 'major occupation code',
       'race', 
         #'hispanic origin', 
         'gender', 
#          'member of a labor union',
#        'reason for unemployment', 'full or part time employment stat',
#        'capital gains', 'capital losses', 'divdends from stocks',
#        'tax filer status', 'region of previous residence',
#        'state of previous residence', 'detailed household and family stat',
#        'detailed household summary in household',
#        'migration code-change in msa', 'migration code-change in reg',
#        'migration code-move within reg', 'live in this house 1 year ago',
#        'migration prev res in sunbelt', 'num persons worked for employer',
#        'family members under 18', 'country of birth father',
#        'country of birth mother', 'country of birth self', 
         'citizenship',
#        'own business or self employed',
#        'fill inc questionnaire for veteran\'s admin', 'veterans benefits',
#        'weeks worked in year', 'YEAR', 
         'TARGET']].copy()

In [19]:
# df.loc[df.duplicated()]
# # Check for duplicate coaster name
# df.loc[df.duplicated(subset=['Coaster_Name'])].head(5)

# Step 3: Feature Understanding
(Univariate analysis)

- Plotting Feature Distributions
    - Histogram
    - KDE
    - Boxplot

# Step 4: Feature Relationships
- Scatterplot
- Heatmap Correlation
- Pairplot
- Groupby comparisons